# Elhub API data - Gridloss - Summerproject 2025

## Visualization of Price area

Bjørn Eirik Rognskog Nordbak

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

oslo = ZoneInfo("Europe/Oslo")

def fetch_window(start_dt, end_dt):
    params = {
        "dataset":   "LOSS_PER_MBA_HOUR",
        "startDate": start_dt.isoformat(),
        "endDate":   end_dt.isoformat(),
    }
    url = "https://api.elhub.no/energy-data/v0/price-areas"
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    raw = resp.json().get("data", [])
    if not raw:
        print(f"  → no data for {start_dt.date()} → {end_dt.date()}, skipping")
        return pd.DataFrame()

    df = pd.json_normalize(
        raw,
        record_path=["attributes", "lossPerMbaHour"],
        meta=[
            ["attributes", "eic"],
            ["attributes", "name"],
            ["attributes", "status"],
        ],
        errors="ignore"
    ).rename(columns={
        "attributes.eic":    "eic",
        "attributes.name":   "name",
        "attributes.status": "status",
    })

    # parse as UTC, then convert to Oslo
    df["startTime"]       = pd.to_datetime(df["startTime"],       utc=True).dt.tz_convert(oslo)
    df["endTime"]         = pd.to_datetime(df["endTime"],         utc=True).dt.tz_convert(oslo)
    df["lastUpdatedTime"] = pd.to_datetime(df["lastUpdatedTime"], utc=True).dt.tz_convert(oslo)

    return df

# loop exactly as before
span_start = datetime(2022, 1, 1, tzinfo=oslo)
span_end   = datetime(2025, 7, 1, tzinfo=oslo)
window     = timedelta(days=7)

all_chunks = []
cur = span_start
while cur < span_end:
    nxt = min(cur + window, span_end)
    print(f"Fetching {cur.date()} → {nxt.date()}")
    all_chunks.append(fetch_window(cur, nxt))
    cur = nxt

big_df = pd.concat(all_chunks, ignore_index=True)
print("Total rows fetched:", len(big_df))


In [ ]:
big_df

## Kraftpriser i sluttbrukermarkedet

In [ ]:
import pandas as pd
import csv
import os

# Sett filnavnet (endre om nødvendig)
filename = 'kraftpriser_historisk.csv'
# Hvis du ikke er sikker på om det er .csv, kan du også bare 'kraftpriser_historisk'

# Sjekk at filen finnes
if not os.path.exists(filename):
    raise FileNotFoundError(f"Finner ikke filen '{filename}' i gjeldende katalog.")

# Les et lite utdrag for å finne separator
with open(filename, 'r', encoding='utf-8') as f:
    sample = f.read(2048)  # les litt mer for å være sikker
dialect = csv.Sniffer().sniff(sample)
sep = dialect.delimiter
print(f"Oppdaget separator: '{sep}'\n")

# Les hele CSV‐filen med riktig separator til df_kraftpris
df_kraftpris = pd.read_csv(filename, sep=sep, encoding='utf-8')

display(df_kraftpris)

# Vis grunnleggende info om DataFrame
print("\nInformasjon om df_kraftpris:")
df_kraftpris.info()


### Fetching consumption MBA data from Elhub energy data API

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

# Set Oslo timezone
oslo = ZoneInfo("Europe/Oslo")

def fetch_window(start_dt, end_dt):
    params = {
        "dataset":   "CONSUMPTION_PER_GROUP_MBA_HOUR",  # Dataset for consumption
        "startDate": start_dt.isoformat(),
        "endDate":   end_dt.isoformat(),
    }
    url = "https://api.elhub.no/energy-data/v0/price-areas"
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    raw = resp.json().get("data", [])
    if not raw:
        print(f"  → no data for {start_dt.date()} → {end_dt.date()}, skipping")
        return pd.DataFrame()

    # Normalize the JSON structure for consumption data
    df = pd.json_normalize(
        raw,
        record_path=["attributes", "consumptionPerGroupMbaHour"],
        meta=[
            ["attributes", "eic"],
            ["attributes", "name"],
            ["attributes", "status"],
        ],
        errors="ignore"
    ).rename(columns={
        "attributes.eic":    "eic",
        "attributes.name":   "name",
        "attributes.status": "status",
    })

    # Parse timestamps as UTC, then convert to Oslo
    df["startTime"]       = pd.to_datetime(df["startTime"],       utc=True).dt.tz_convert(oslo)
    df["endTime"]         = pd.to_datetime(df["endTime"],         utc=True).dt.tz_convert(oslo)
    df["lastUpdatedTime"] = pd.to_datetime(df["lastUpdatedTime"], utc=True).dt.tz_convert(oslo)

    return df

# Define the overall fetch window: Jan 1, 2022 → Jul 1, 2025, in 7-day increments
span_start = datetime(2022, 1, 1, tzinfo=oslo)
span_end   = datetime(2025, 7, 1, tzinfo=oslo)
window     = timedelta(days=7)

all_chunks = []
cur = span_start
while cur < span_end:
    nxt = min(cur + window, span_end)
    print(f"Fetching {cur.date()} → {nxt.date()}")
    all_chunks.append(fetch_window(cur, nxt))
    cur = nxt

# Concatenate all resulting DataFrames
consumption_mba = pd.concat(all_chunks, ignore_index=True)
print("Total rows fetched:", len(consumption_mba))


In [ ]:
consumption_mba

In [ ]:
# Hent alle unike grupper
unike_grupper = consumption_mba["consumptionGroup"].unique()

# Skriv dem ut
print("Unike consumptionGroup-verdier:")
for grp in unike_grupper:
    print(" –", grp)


## Data quality

### Data quality report

In [ ]:
import pandas as pd

def humanized_dq_report(df):
    # base stats
    present     = df.notnull().sum()
    missing     = df.isnull().sum()
    pct_missing = (missing / len(df) * 100).round(1)
    unique_vals = df.nunique(dropna=False)
    dtypes      = df.dtypes.astype(str)
    
    # map raw dtypes to friendly types
    def friendly_dtype(dt):
        if "float" in dt or "int" in dt:
            return "Numeric"
        if "datetime" in dt:
            return "Date/Time"
        return "Text"
    
    # build the report without Range
    report = pd.DataFrame({
        "Column":    present.index,
        "Type":      [friendly_dtype(d) for d in dtypes],
        "Present":   present.values,
        "Missing":   missing.values,
        "% Missing": pct_missing.values,
        "Unique":    unique_vals.values
    })
    
    # reorder for readability (Range removed)
    cols = ["Column", "Type", "Present", "Missing", "% Missing", "Unique"]
    return report[cols]

# generate it
dq = humanized_dq_report(big_df)

# blank out the index so it doesn’t show up
dq_display = dq.copy()
dq_display.index = [""] * len(dq_display)

# display with nice formatting
fmt = {
    "Present":   "{:,}",
    "Missing":   "{:,}",
    "% Missing": "{:.1f}%",
    "Unique":    "{:,}"
}

dq_display.style \
    .format(fmt) \
    .set_caption(f"Data Quality Summary ({len(big_df):,} rows × {big_df.shape[1]} columns)") \
    .set_table_styles([
        {
            "selector": "caption",
            "props": [
                ("caption-side","bottom"),
                ("font-style","italic"),
                ("text-align","left"),
            ]
        }
    ])


### Save the data quality report to a .tex table

In [ ]:
import os

# 1) ensure the output folder exists
os.makedirs("tables", exist_ok=True)

# 2) copy & format exactly as before
dq_tex = dq.copy()
dq_tex["Present"]    = dq_tex["Present"].map("{:,}".format)
dq_tex["Missing"]    = dq_tex["Missing"].map("{:,}".format)
dq_tex["% Missing"]  = dq_tex["% Missing"].map("{:.1f}\\%".format)
dq_tex["Unique"]     = dq_tex["Unique"].map("{:,}".format)

# 3) rename the column header so it's "\% Missing" instead of "% Missing"
dq_tex = dq_tex.rename(columns={"% Missing": r"\% Missing"})

# 4) export _only_ the tabular (no float wrapper, caption, label, etc.)
tabular_str = dq_tex.to_latex(
    index=False,
    longtable=False,  # plain tabular
    caption=None,
    label=None,
    escape=False      # keep our LaTeX markup in the cells
)

# 5) write out to a .tex
out_path = "tables/data_quality_pricearea_tabular.tex"
with open(out_path, "w") as f:
    f.write(tabular_str)

print(f"Wrote tabular-only file to {out_path}")


### Checking for temporal consistency

In [ ]:
import pandas as pd

# 1. Make sure your time columns are in datetime format
df = big_df.copy()
df['startTime'] = pd.to_datetime(df['startTime'])
df['endTime']   = pd.to_datetime(df['endTime'])

# 2. Sort chronologically
df = df.sort_values('startTime').reset_index(drop=True)

# 3. Compute the gap between each start and the previous end
df['prev_end'] = df['endTime'].shift(1)
df['gap']      = df['startTime'] - df['prev_end']

# 4. Filter only the actual positive gaps
gaps = df[df['gap'] > pd.Timedelta(0)].copy()

# 5. Build a summary
num_gaps    = len(gaps)
total_gap   = gaps['gap'].sum()
avg_gap     = gaps['gap'].mean()
max_gap     = gaps['gap'].max()

print(f"🔎 Found {num_gaps} gap(s) in the series")
print(f"⏳ Total missing time: {total_gap}")
print(f"📈 Average gap length: {avg_gap}")
print(f"🏆 Largest gap: {max_gap}")

# 6. (Optional) Inspect the top few largest gaps
top10 = (gaps
         .sort_values('gap', ascending=False)
         .head(10)
         [['prev_end','startTime','gap']]
        )
print("\nTop 10 largest gaps:")
print(top10.to_string(index=False))


## Statistics

### Summary for each price area

In [ ]:
import os
import pandas as pd

# ─── 0) Ensure output folder exists ────────────────────────────────────────────
os.makedirs("tables", exist_ok=True)

# ─── 1) Compute global date range if you want it in captions ──────────────────
start = big_df['endTime'].min().date()
end   = big_df['endTime'].max().date()

# ─── 2) KWh→MWh conversion and human‐friendly column names ────────────────────
kwh_to_mwh = 1e3
col_mapping = {
    'calculateLossFraction':     'Loss fraction',
    'calculatedLossQuantityKwh': 'Losses (MWh)',
    'consumptionQuantityKwh':    'Consumption (MWh)',
    'exchangeInQuantityKwh':     'Exchange In (MWh)',
    'exchangeOutQuantityKwh':    'Exchange Out (MWh)',
    'netInfeedQuantityKwh':      'Net Infeed (MWh)',
    'productionQuantityKwh':     'Production (MWh)',
}

# ─── 3) Loop per priceArea ────────────────────────────────────────────────────
for area, grp in big_df.groupby('priceArea'):
    # 3a) summary stats (drop count)
    desc = grp.describe().drop(index='count')
    
    # 3b) convert energy cols to MWh
    energy_cols = [c for c in desc.columns if c!='calculateLossFraction']
    desc[energy_cols] /= kwh_to_mwh
    
    # 3c) rename columns
    desc = desc.rename(columns=col_mapping)
    
    # 3d) reorder rows
    order = ['mean','25%','50%','75%','min','max']
    desc = desc.loc[order, list(col_mapping.values())]
    
    # 3e) escape % in index
    desc.rename(index=lambda i: i.replace('%', r'\%'), inplace=True)
    
    # ─── 3f) SHOW IN NOTEBOOK ─────────────────────────────────────────────────
    # formatting for display
    disp_fmt = {c:"{:,.0f}" for c in col_mapping.values()}
    disp_fmt['Loss fraction'] = "{:.2%}"
    caption = f"{area} Zone Aggregates, {start} – {end}"
    
    styled = (
        desc.style
            .format(disp_fmt)
            .set_caption(caption)
            .set_table_styles([{
                "selector": "caption",
                "props": [
                    ("caption-side","bottom"),
                    ("font-style","italic"),
                    ("text-align","left")
                ]
            }])
    )
    display(styled)  # <-- keeps your display in Jupyter
    
    # ─── 4) WRITE OUT tabular‐only .tex ────────────────────────────────────────
    # prepare LaTeX‐safe formatters
    def make_fmt(fmt_str):
        return lambda x: fmt_str.format(x).replace('%', r'\%')
    
    latex_fmt = {col: make_fmt(disp_fmt[col]) for col in disp_fmt}
    
    tabular = desc.to_latex(
        formatters=latex_fmt,
        na_rep="–",
        column_format="l" + "r"*desc.shape[1],
        bold_rows=True,
        index_names=False,
        longtable=False,
        escape=False
    )
    
    out_path = os.path.join("tables", f"summary_{area}.tex")
    with open(out_path, "w") as f:
        f.write(tabular)
    print(f"Wrote tabular‐only file for {area}: {out_path}")


## Time series diagnostics

### Seasonal decompostion

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Prepare your DataFrame exactly as before
df = big_df.copy()
df['startTime'] = pd.to_datetime(df['startTime'])
df = df.set_index('startTime').sort_index()

# 2. Pick zone and series
zone = 'NO1'
ts = df.loc[df['priceArea']==zone, 'consumptionQuantityKwh'].asfreq('H')

# 3. Trend via centered 24-h rolling average
trend = ts.rolling(window=24, center=True, min_periods=12).mean()

# 4. Detrend
detrended = ts - trend

# 5. Seasonal: average detrended by hour‐of‐day
#    (you could also do day-of-year, week-of-year, etc.)
seasonal_lookup = detrended.groupby(detrended.index.hour).mean()
seasonal = detrended.index.map(lambda t: seasonal_lookup[t.hour])
seasonal = pd.Series(seasonal, index=ts.index)

# 6. Residual
resid = ts - trend - seasonal

# 7. Plot
for name, series in [('Observed', ts),
                     ('Trend', trend),
                     ('Seasonal (daily)', seasonal),
                     ('Residual', resid)]:
    plt.figure()
    plt.plot(series)
    plt.title(f'{zone} Consumption – {name}')
    plt.xlabel('Time')
    plt.ylabel('kWh')
    plt.tight_layout()

plt.show()


## Visualizations

### Scatter plot of production vs. consumption

In [ ]:
import os
import pandas as pd
import plotly.express as px

# ─── MAKE OUTPUT FOLDER ───────────────────────────────────────────────────────
OUTDIR = "figures"
os.makedirs(OUTDIR, exist_ok=True)

# ─── FONT & COLOR SETTINGS ───────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
FONT_FAMILY   = "Roboto"

elhub_colors = [
    "#212148",  # Mørk Lilla
    "#7fb48a",  # Grønn
    "#5369b2",  # Blå
    "#05677d",  # Blå-grønn
    "#886599",  # Lilla
    "#d58000",  # Oker
]

# ─── PREPARE DATA ─────────────────────────────────────────────────────────────
df = big_df.copy()
df['startTime'] = (
    pd.to_datetime(df['startTime'], utc=True)
      .dt.tz_convert('Europe/Oslo')
)

span_start = df['startTime'].min().date().isoformat()
span_end   = df['startTime'].max().date().isoformat()
title_base = f"Consumption vs Production (kWh) per Price Area\n({span_start} to {span_end})"

MAX_POINTS = 5000
def sample_group(g):
    return g.sample(n=min(len(g), MAX_POINTS), random_state=1)

areas = df['priceArea'].unique().tolist()

# Compute global production range for axis
prod_min = df['productionQuantityKwh'].min()
prod_max = df['productionQuantityKwh'].max()

# ─── PER-AREA SCATTERS ────────────────────────────────────────────────────────
for area in areas:
    dfa = sample_group(df[df['priceArea'] == area])
    idx = areas.index(area)
    color = elhub_colors[idx % len(elhub_colors)]
    
    fig = px.scatter(
        dfa,
        x='productionQuantityKwh',
        y='consumptionQuantityKwh',
        title=f"{title_base} — {area}",
        labels={
            'productionQuantityKwh': 'Production (kWh)',
            'consumptionQuantityKwh': 'Consumption (kWh)'
        },
        color_discrete_sequence=[color]
    )
    
    fig.update_xaxes(range=[prod_min, prod_max])
    fig.update_layout(
        title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
        title_x=0.5,
        font=dict(family=FONT_FAMILY),
        xaxis=dict(
            title='Production (kWh)',
            title_font=dict(size=AXIS_TITLE_FS),
            tickfont=dict(size=TICK_FS)
        ),
        yaxis=dict(
            title='Consumption (kWh)',
            title_font=dict(size=AXIS_TITLE_FS),
            tickfont=dict(size=TICK_FS)
        ),
        margin=dict(t=100, b=80),
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    fig.show()
    
    fname = f"scatter_loss_per_mba_hour_{area.lower()}.pdf"
    path  = os.path.join(OUTDIR, fname)
    fig.write_image(path, format="pdf", width=1000, height=600, scale=1)
    print(f"✅ Saved: {path}")

# ─── COMBINED SCATTER ─────────────────────────────────────────────────────────
df_sample = df.groupby('priceArea', group_keys=False).apply(sample_group)

fig_all = px.scatter(
    df_sample,
    x='productionQuantityKwh',
    y='consumptionQuantityKwh',
    color='priceArea',
    title=f"{title_base} — All Areas",
    labels={
        'productionQuantityKwh': 'Production (kWh)',
        'consumptionQuantityKwh': 'Consumption (kWh)'
    },
    color_discrete_sequence=elhub_colors
)

fig_all.update_xaxes(range=[prod_min, prod_max])
fig_all.update_layout(
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    font=dict(family=FONT_FAMILY),
    xaxis=dict(
        title='Production (kWh)',
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    yaxis=dict(
        title='Consumption (kWh)',
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    margin=dict(t=100, b=80),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_all.show()

combined_fname = "scatter_loss_per_mba_hour_all.pdf"
combined_path  = os.path.join(OUTDIR, combined_fname)
fig_all.write_image(combined_path, format="pdf", width=1000, height=600, scale=1)
print(f"✅ Saved: {combined_path}")


### Scatter plot of production vs consumption - with correlation line and R2 score

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# ─── MAKE OUTPUT FOLDER ───────────────────────────────────────────────────────
OUTDIR = "figures"
os.makedirs(OUTDIR, exist_ok=True)

# ─── FONT & COLOR SETTINGS ───────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
FONT_FAMILY   = "Roboto"

elhub_colors = [
    "#212148",  # Mørk Lilla
    "#7fb48a",  # Grønn
    "#5369b2",  # Blå
    "#05677d",  # Blå-grønn
    "#886599",  # Lilla
    "#d58000",  # Oker
]

# ─── NAMING PREFIX ─────────────────────────────────────────────────────────────
FILENAME_PREFIX = "scatter_loss_per_mba_hour_prod_consum"

# ─── PREPARE DATA ─────────────────────────────────────────────────────────────
df = big_df.copy()
df['startTime'] = (
    pd.to_datetime(df['startTime'], utc=True)
      .dt.tz_convert('Europe/Oslo')
)

span_start = df['startTime'].min().date().isoformat()
span_end   = df['startTime'].max().date().isoformat()
title_base = f"Consumption vs Production (kWh) per Price Area\n({span_start} to {span_end})"

MAX_POINTS = 5000
def sample_group(g):
    return g.sample(n=min(len(g), MAX_POINTS), random_state=1)

areas = df['priceArea'].unique().tolist()

# Global prod range
prod_min, prod_max = df['productionQuantityKwh'].min(), df['productionQuantityKwh'].max()

def make_regression_trace(x, y, color):
    """Fit linear model, return line trace and R²."""
    # reshape for sklearn
    X = x.values.reshape(-1, 1)
    Y = y.values
    model = LinearRegression().fit(X, Y)
    r2 = model.score(X, Y)
    
    # create line points
    xs = np.array([prod_min, prod_max])
    ys = model.predict(xs.reshape(-1,1))
    
    line = go.Scatter(
        x=xs, y=ys,
        mode='lines',
        line=dict(color=color, dash='dash'),
        name=f"Fit (R²={r2:.2f})",
        hoverinfo='none'
    )
    return line, r2

# ─── PER-AREA PLOTS ───────────────────────────────────────────────────────────
for area in areas:
    dfa = sample_group(df[df['priceArea'] == area])
    idx = areas.index(area)
    color = elhub_colors[idx % len(elhub_colors)]
    
    # Build scatter
    scatter = go.Scatter(
        x=dfa['productionQuantityKwh'],
        y=dfa['consumptionQuantityKwh'],
        mode='markers',
        marker=dict(color=color, size=6, opacity=0.6),
        name=area
    )
    
    # Regression
    line, r2 = make_regression_trace(
        dfa['productionQuantityKwh'],
        dfa['consumptionQuantityKwh'],
        color
    )
    
    fig = go.Figure([scatter, line])
    fig.update_layout(
        title=f"{title_base} — {area} (R²={r2:.2f})",
        title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
        title_x=0.5,
        xaxis=dict(
            title='Production (kWh)',
            range=[prod_min, prod_max],
            title_font=dict(size=AXIS_TITLE_FS),
            tickfont=dict(size=TICK_FS)
        ),
        yaxis=dict(
            title='Consumption (kWh)',
            title_font=dict(size=AXIS_TITLE_FS),
            tickfont=dict(size=TICK_FS)
        ),
        font=dict(family=FONT_FAMILY),
        margin=dict(t=120, b=80),
        plot_bgcolor='white',
        paper_bgcolor='white',
        showlegend=False
    )
    
    fig.show()
    
    fname = f"{FILENAME_PREFIX}_{area.lower()}.pdf"
    path  = os.path.join(OUTDIR, fname)
    fig.write_image(path, format="pdf", width=1000, height=600, scale=1)
    print(f"✅ Saved: {path}")

# ─── COMBINED PLOT ────────────────────────────────────────────────────────────
df_samp = df.groupby('priceArea', group_keys=False).apply(sample_group)

# Scatter traces for each area
traces = []
for idx, area in enumerate(areas):
    dfa = df_samp[df_samp['priceArea'] == area]
    col = elhub_colors[idx % len(elhub_colors)]
    traces.append(
        go.Scatter(
            x=dfa['productionQuantityKwh'],
            y=dfa['consumptionQuantityKwh'],
            mode='markers',
            marker=dict(color=col, size=6, opacity=0.6),
            name=area
        )
    )

# Global regression on combined sample
comb_line, comb_r2 = make_regression_trace(
    df_samp['productionQuantityKwh'],
    df_samp['consumptionQuantityKwh'],
    '#000000'  # black for combined fit line
)
traces.append(comb_line)

fig_all = go.Figure(traces)
fig_all.update_layout(
    title=f"{title_base} — All Areas (R²={comb_r2:.2f})",
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    xaxis=dict(
        title='Production (kWh)',
        range=[prod_min, prod_max],
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    yaxis=dict(
        title='Consumption (kWh)',
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    font=dict(family=FONT_FAMILY),
    margin=dict(t=120, b=80),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_all.show()

combined_fname = f"{FILENAME_PREFIX}_all.pdf"
combined_path  = os.path.join(OUTDIR, combined_fname)
fig_all.write_image(combined_path, format="pdf", width=1000, height=600, scale=1)
print(f"✅ Saved: {combined_path}")


### Scatter plot of loss fraction vs. price area

### boxplot of loss fraction by price area

In [ ]:
import matplotlib.pyplot as plt

# 1) group your loss fractions by area
groups = [
    grp['calculateLossFraction'].values
    for _, grp in big_df.groupby('priceArea')
]
labels = [
    name
    for name, _ in big_df.groupby('priceArea')
]

# 2) draw the boxplot
plt.figure(figsize=(12, 6))
plt.boxplot(groups, labels=labels, notch=True, patch_artist=True)

# 3) add a horizontal zero-loss line
plt.axhline(0, linestyle='--', linewidth=1, color='gray')

plt.xlabel('Price Area')
plt.ylabel('Loss Fraction')
plt.title('Distribution of Loss Fraction by Price Area')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('figures/loss_fraction_by_price_area_boxplot.pdf', format='pdf')
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt

# … group your data into `groups` and `labels` as before …

plt.figure(figsize=(12, 6))
plt.boxplot(
    groups,
    labels=labels,
    notch=True,
    patch_artist=True,
    whis=1.5,
    showfliers=False
)
plt.axhline(0, linestyle='--', linewidth=1, color='gray')
plt.xlabel('Price Area')
plt.ylabel('Loss Fraction')
plt.title('Distribution of Loss Fraction by Price Area (no outliers)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# save as PDF
os.makedirs('figures', exist_ok=True)
plt.savefig('figures/loss_fraction_by_price_area_no_outliers_boxplot.pdf', format='pdf')

plt.show()


### Violin plot of loss fractions by price area

### Violin plot of loss fractions by price area - cleaned outliers

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def remove_outliers_iqr(df, value_col, group_col):
    """
    Remove rows where value_col is outside [Q1 - 1.5*IQR, Q3 + 1.5*IQR]
    calculated within each group of group_col.
    """
    def _filter(group):
        q1 = group[value_col].quantile(0.25)
        q3 = group[value_col].quantile(0.75)
        iqr = q3 - q1
        lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
        return group[(group[value_col] >= lower) & (group[value_col] <= upper)]
    
    # Apply per-group filtering and re-concatenate
    return df.groupby(group_col, group_keys=False).apply(_filter)

# 1) Filter out outliers
clean_df = remove_outliers_iqr(
    big_df,
    value_col='calculateLossFraction',
    group_col='priceArea'
)

# 2) Plot the violin plot on the cleaned data
plt.figure(figsize=(12, 6))
sns.violinplot(
    x='priceArea',
    y='calculateLossFraction',
    data=clean_df,
    hue='priceArea',         # allows using a pastel palette
    inner='quartile',         # show quartile lines
    cut=0,                    # no density beyond data range
    density_norm='width',     # uniform max width
    palette='pastel',
    dodge=False,
    legend=False
)

# 3) Zero-loss reference line
plt.axhline(0, linestyle='--', linewidth=1, color='gray')

# 4) Labels & layout
plt.xlabel('Price Area')
plt.ylabel('Loss Fraction')
plt.title('Violin Plot of Loss Fraction by Price Area\n(after removing outliers)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('figures/loss_fraction_by_price_area_no_outliers_violin.pdf', format='pdf')
plt.show()


### Heatmap of cross correlation between numerical columns

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) your raw column names
numeric_cols = [
    'calculateLossFraction',
    'calculatedLossQuantityKwh',
    'consumptionQuantityKwh',
    'exchangeInQuantityKwh',
    'exchangeOutQuantityKwh',
    'netInfeedQuantityKwh',
    'productionQuantityKwh',
]

# 2) the corresponding “nice” labels
nice_labels = [
    'Loss fraction',
    'Losses (MWh)',
    'Consumption (MWh)',
    'Exchange In (MWh)',
    'Exchange Out (MWh)',
    'Net Infeed (MWh)',
    'Production (MWh)',
]

# compute correlation
corr = big_df[numeric_cols].corr()

# mask out the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool), k=1)
masked_corr = np.ma.masked_array(corr, mask=mask)

# grab the 'bwr' cmap and set masked entries to white
cmap = plt.cm.bwr
cmap.set_bad(color='white')

fig, ax = plt.subplots(figsize=(8, 6))
cax = ax.imshow(masked_corr, cmap=cmap, vmin=-1, vmax=1)

# set ticks & labels
ax.set_xticks(np.arange(len(nice_labels)))
ax.set_yticks(np.arange(len(nice_labels)))
ax.set_xticklabels(nice_labels, rotation=45, ha='right')
ax.set_yticklabels(nice_labels)

# annotate only the visible (lower-triangle + diagonal) cells
for i in range(len(numeric_cols)):
    for j in range(len(numeric_cols)):
        if not mask[i, j]:
            val = corr.iloc[i, j]
            color = 'white' if abs(val) > 0.5 else 'black'
            ax.text(j, i, f"{val:.2f}", ha='center', va='center', color=color)

# add colorbar
fig.colorbar(cax, ax=ax, fraction=0.046, pad=0.04, label='Pearson r')

plt.tight_layout()
plt.show()


### Daglig nettap profil - prisområde

In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

# ─── VISNING I NOTEBOOK ─────────────────────────────────────────────────────────
pio.renderers.default = "notebook"  # eller "jupyterlab" hvis du er i JupyterLab

# ─── FONT- OG STILINNSTILLINGER ────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
MARKER_SIZE   = 8
FONT_FAMILY   = "Roboto"
# ──────────────────────────────────────────────────────────────────────────────

# Elhub sine farger
elhub_colors = [
    "#212148",  # mørklilla
    "#7fb48a",  # grønn
    "#5369b2",  # blå
    "#05677d",  # blå-grønn
    "#886599",  # lilla
    "#d58000",  # oker
]

# ─── 0) Klargjør dataframe ─────────────────────────────────────────────────────
df = big_df.copy()

# 1) Finn tidskolonne
time_col = "startTime" if "startTime" in df.columns else "endTime"

# 2) Finn områdekolonne
area_col = "priceArea" if "priceArea" in df.columns else "name"

# 3) Finn tapsmengde-kolonne (fall-back til consumption eller tapsfraksjon)
if "calculatedLossQuantityKwh" in df.columns:
    value_col = "calculatedLossQuantityKwh"
elif "consumptionQuantityKwh" in df.columns:
    value_col = "consumptionQuantityKwh"
else:
    value_col = "calculateLossFraction"

# 4) Konverter til datetime + Oslo-sone
df[time_col] = (
    pd.to_datetime(df[time_col], utc=True)
      .dt.tz_convert("Europe/Oslo")
)

# 5) Dato-span for tittel
span_start = df[time_col].min().date().isoformat()
span_end   = df[time_col].max().date().isoformat()

# 6) Ekstraher time på døgnet
df["timePaaDøgnet"] = df[time_col].dt.hour

# 7) Beregn gjennomsnittlig tap per time og område (i kWh)
avg_hourly = (
    df
    .groupby(["timePaaDøgnet", area_col], observed=True)[value_col]
    .mean()
    .reset_index()
    .rename(columns={value_col: "gjennomsnittligTimevisTapKWh"})
)

# 8) Konverter kWh → MWh
avg_hourly["gjennomsnittligTimevisTapMWh"] = avg_hourly["gjennomsnittligTimevisTapKWh"] / 1_000

# 9) Beregn andel av totalsum per time
tot_per_time = avg_hourly.groupby("timePaaDøgnet")["gjennomsnittligTimevisTapMWh"].transform("sum")
avg_hourly["andelAvTapetDenTimen"] = avg_hourly["gjennomsnittligTimevisTapMWh"] / tot_per_time

# ─── 9b) Bygg oppsummerings-data for tabell uten median ────────────────────────
summary = []
for area, grp in avg_hourly.groupby(area_col):
    i_max  = grp["gjennomsnittligTimevisTapMWh"].idxmax()
    i_min  = grp["gjennomsnittligTimevisTapMWh"].idxmin()
    t_max  = f"{int(grp.at[i_max,'timePaaDøgnet']):02d}:00"
    v_max  = grp.at[i_max, "gjennomsnittligTimevisTapMWh"]
    t_min  = f"{int(grp.at[i_min,'timePaaDøgnet']):02d}:00"
    v_min  = grp.at[i_min, "gjennomsnittligTimevisTapMWh"]
    mean   = grp["gjennomsnittligTimevisTapMWh"].mean()
    std    = grp["gjennomsnittligTimevisTapMWh"].std()
    amp    = v_max - v_min
    total  = grp["gjennomsnittligTimevisTapMWh"].sum()

    summary.append({
        "Sone":        area,
        "Tid maks.":   t_max,
        "Maks (MWh)":  v_max,
        "Tid min.":    t_min,
        "Min (MWh)":   v_min,
        "Snitt":       mean,
        "St.avv.":     std,
        "Amp. (MWh)":  amp,
        "Tot. (MWh)":  total,
    })

summary_df = pd.DataFrame(summary)

# Runder alle tall-kolonner til 1 desimal
for col in ["Maks (MWh)", "Min (MWh)", "Snitt", "St.avv.", "Amp. (MWh)", "Tot. (MWh)"]:
    summary_df[col] = summary_df[col].round(1)

# ─── 9c) Lagre kun tabular-delen med korte overskrifter ───────────────────────
os.makedirs("tables", exist_ok=True)
out_path = os.path.join("tables", "summary_timewise_loss.tex")

tabular_only = summary_df.to_latex(
    index=False,
    float_format="%.1f",
    escape=False,
    column_format="l" + "r"*(len(summary_df.columns)-1)
)

with open(out_path, "w", encoding="utf-8") as f:
    f.write(tabular_only)

print(f"✅ Lagret tabular-only til: {out_path}")
display(summary_df)

# ─── 10) Lag linjeplott (med de opprinnelige, lengre etikettene) ──────────────
tittel = f"Gjennomsnittlig timevis tap per prisområde\n{span_start} til {span_end}"
fig = px.line(
    avg_hourly,
    x="timePaaDøgnet",
    y="gjennomsnittligTimevisTapMWh",
    color=area_col,
    custom_data=[area_col, "andelAvTapetDenTimen"],
    markers=True,
    title=tittel,
    labels={
        "timePaaDøgnet": "Time på døgnet",
        "gjennomsnittligTimevisTapMWh": "Gjennomsnittlig timevis tap (MWh)"
    },
    color_discrete_sequence=elhub_colors
)
fig.update_traces(
    marker=dict(size=MARKER_SIZE),
    line=dict(width=3),
    hovertemplate=(
        "%{y:.2f} MWh<br>"
        "Time på døgnet: %{x}<br>"
        "Prisområde: %{customdata[0]}<br>"
        "Andel av tapet den timen: %{customdata[1]:.1%}<extra></extra>"
    )
)
fig.update_layout(
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    font=dict(family=FONT_FAMILY),
    xaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS),
        dtick=1
    ),
    yaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    legend_title_text="Prisområde",
    legend=dict(
        title_font_size=AXIS_TITLE_FS,
        font_size=TICK_FS,
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    margin=dict(t=100, b=80),
    plot_bgcolor="white",
    paper_bgcolor="white"
)
fig.show()

# ─── 11) Lagre figuren ─────────────────────────────────────────────────────────
os.makedirs("figures", exist_ok=True)
pdf_path = os.path.join("figures", "diurnaltap_prisområde_MWh.pdf")
fig.write_image(pdf_path, format="pdf", width=1000, height=700, scale=1)
print(f"✅ Lagret PDF til: {pdf_path}")


### Økonomisk analyse av tap

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# ─── VISNING I NOTEBOOK ─────────────────────────────────────────────────────────
pio.renderers.default = "notebook"  # eller "jupyterlab" hvis du er i JupyterLab

# ─── FONT- OG STILINNSTILLINGER ────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
MARKER_SIZE   = 8
FONT_FAMILY   = "Roboto"

# Elhub sine farger
elhub_colors = [
    "#212148",  # mørklilla
    "#7fb48a",  # grønn
    "#5369b2",  # blå
    "#05677d",  # blå-grønn
    "#886599",  # lilla
    "#d58000",  # oker
]

# ─── KLARGJØR DATA ──────────────────────────────────────────────────────────────
# Antar df_econ med kolonner "Sone", "Kr/uke_vinter", etc.
df_plot = df_econ.melt(
    id_vars='Sone',
    value_vars=['Kr/uke_vinter','Kr/uke_vår','Kr/uke_sommer','Kr/uke_høst'],
    var_name='Sesong',
    value_name='tap_kr_uke'
)
df_plot['Sesong'] = df_plot['Sesong'].str.replace('Kr/uke_', '')
df_plot['Sesong'] = pd.Categorical(
    df_plot['Sesong'], 
    categories=['vinter','vår','sommer','høst'], 
    ordered=True
)

# ─── LAG INTERAKTIVT PLOT ──────────────────────────────────────────────────────
fig = px.line(
    df_plot,
    x='Sesong',
    y='tap_kr_uke',
    color='Sone',
    markers=True,
    title='Ukentlig økonomisk tap per sesong og prisområde',
    labels={'tap_kr_uke': 'Ukentlig tap (kr)', 'Sesong': 'Sesong'},
    color_discrete_sequence=elhub_colors
)
fig.update_traces(
    marker=dict(size=MARKER_SIZE),
    line=dict(width=3)
)
fig.update_layout(
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    font=dict(family=FONT_FAMILY),
    xaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    yaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    legend_title_text="Prisområde",
    legend=dict(
        title_font_size=AXIS_TITLE_FS,
        font_size=TICK_FS,
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    margin=dict(t=100, b=80),
    plot_bgcolor="white",
    paper_bgcolor="white"
)

fig.show()


### Daglig nettap profil i andel - prisområde

In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

# ─── VISNING I NOTEBOOK ─────────────────────────────────────────────────────────
pio.renderers.default = "notebook"  # eller "jupyterlab" hvis du er i JupyterLab
# ──────────────────────────────────────────────────────────────────────────────

# ─── FONT- OG STILINNSTILLINGER ────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
MARKER_SIZE   = 8
FONT_FAMILY   = "Roboto"
# ──────────────────────────────────────────────────────────────────────────────

# Elhub sine farger
elhub_colors = [
    "#212148",  # Mørk lilla
    "#7fb48a",  # Grønn
    "#5369b2",  # Blå
    "#05677d",  # Blå-grønn
    "#886599",  # Lilla
    "#d58000",  # Oker
]

# ─── 0) Klargjør dataframe ─────────────────────────────────────────────────────
df = big_df.copy()

# 1) Finn tidskolonne
time_col = "startTime" if "startTime" in df.columns else "endTime"

# 2) Finn områdekolonne
area_col = "priceArea" if "priceArea" in df.columns else "name"

# 3) Bruk tapsfraksjons-kolonnen
value_col = "calculateLossFraction"

# 4) Sørg for korrekt datetime og Oslo-sone
df[time_col] = (
    pd.to_datetime(df[time_col], utc=True)
      .dt.tz_convert("Europe/Oslo")
)

# 5) Dato-span for tittel
span_start = df[time_col].min().date().isoformat()
span_end   = df[time_col].max().date().isoformat()

# 6) Ekstraher time på døgnet
df["timePaaDøgnet"] = df[time_col].dt.hour

# 7) Beregn gjennomsnittlig tapsfraksjon per time og område
avg_hourly = (
    df
    .groupby(["timePaaDøgnet", area_col], observed=True)[value_col]
    .mean()
    .reset_index()
    .rename(columns={value_col: "gjennomsnittligTimevisTapsfraksjon"})
)

# 8) Konverter fraksjon til prosent for y-akse
avg_hourly["gj.timevisTapsfraksjonProsent"] = avg_hourly["gjennomsnittligTimevisTapsfraksjon"] * 100

# 9) Andel av totalsum per time (fordeling mellom prisområder)
tot_per_time = avg_hourly.groupby("timePaaDøgnet")["gj.timevisTapsfraksjonProsent"].transform("sum")
avg_hourly["andelAvFraksjonenDenTimen"] = avg_hourly["gj.timevisTapsfraksjonProsent"] / tot_per_time

# 10) Norsk tittel
tittel = (
    f"Gjennomsnittlig timevis tapsfraksjon per prisområde\n"
    f"{span_start} til {span_end}"
)

# 11) Lag linjeplottet
fig = px.line(
    avg_hourly,
    x="timePaaDøgnet",
    y="gj.timevisTapsfraksjonProsent",
    color=area_col,
    custom_data=[area_col, "andelAvFraksjonenDenTimen"],
    markers=True,
    title=tittel,
    labels={
        "timePaaDøgnet": "Time på døgnet",
        "gj.timevisTapsfraksjonProsent": "Gjennomsnittlig tapsfraksjon (%)"
    },
    color_discrete_sequence=elhub_colors
)

# 12) Stil traces og hover-tekst på norsk
fig.update_traces(
    marker=dict(size=MARKER_SIZE),
    line=dict(width=3),
    hovertemplate=(
        "Tapsfraksjon: %{y:.2f}%<br>"
        "Time på døgnet: %{x}<br>"
        "Prisområde: %{customdata[0]}<br>"
        "Andel av tapsfraksjonen den timen: %{customdata[1]:.1%}"
        "<extra></extra>"
    )
)

# 13) Layout-innstillinger
fig.update_layout(
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    font=dict(family=FONT_FAMILY),
    xaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS),
        dtick=1
    ),
    yaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    legend_title_text="Prisområde",
    legend=dict(
        title_font_size=AXIS_TITLE_FS,
        font_size=TICK_FS,
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    margin=dict(t=100, b=80),
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# 14) Vis plottet inline
fig.show()

# 15) Lagre i figures/
os.makedirs("figures", exist_ok=True)
output_path = os.path.join("figures", "diurnalt_tapsfraksjon_prisområde_prosent.pdf")
fig.write_image(output_path, format="pdf", width=1000, height=700, scale=1)
print(f"✅ Lagret PDF til: {output_path}")


### Daglig konsum profil - prisområde

In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

# ─── VISNING I NOTEBOOK ─────────────────────────────────────────────────────────
pio.renderers.default = "notebook"  # eller "jupyterlab" hvis du er i JupyterLab
# ──────────────────────────────────────────────────────────────────────────────

# ─── FONT- OG STILINNSTILLINGER ────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
MARKER_SIZE   = 8
FONT_FAMILY   = "Roboto"
# ──────────────────────────────────────────────────────────────────────────────

# Elhub sine farger
elhub_colors = [
    "#212148",  # Mørk lilla
    "#7fb48a",  # Grønn
    "#5369b2",  # Blå
    "#05677d",  # Blå-grønn
    "#886599",  # Lilla
    "#d58000",  # Oker
]

# ─── 0) Klargjør dataframe ─────────────────────────────────────────────────────
df = big_df.copy()

# 1) Finn tidskolonne
time_col = "startTime" if "startTime" in df.columns else "endTime"

# 2) Finn områdekolonne
area_col = "priceArea" if "priceArea" in df.columns else "name"

# 3) Bruk forbrukskolonnen
value_col = "consumptionQuantityKwh"

# 4) Sørg for korrekt datetime og Oslo-sone
df[time_col] = (
    pd.to_datetime(df[time_col], utc=True)
      .dt.tz_convert("Europe/Oslo")
)

# 5) Dato-span for tittel
span_start = df[time_col].min().date().isoformat()
span_end   = df[time_col].max().date().isoformat()

# 6) Ekstraher time på døgnet
df["timePaaDøgnet"] = df[time_col].dt.hour

# 7) Beregn gjennomsnittlig forbruk per time og område (i kWh)
avg_hourly = (
    df
    .groupby(["timePaaDøgnet", area_col], observed=True)[value_col]
    .mean()
    .reset_index()
    .rename(columns={value_col: "gjennomsnittligTimevisForbrukKWh"})
)

# 8) Konverter til MWh
avg_hourly["gjennomsnittligTimevisForbrukMWh"] = (
    avg_hourly["gjennomsnittligTimevisForbrukKWh"] / 1_000
)

# 9) Andel av totalsum per time
tot_per_time = avg_hourly.groupby("timePaaDøgnet")["gjennomsnittligTimevisForbrukMWh"].transform("sum")
avg_hourly["andelAvForbruketDenTimen"] = avg_hourly["gjennomsnittligTimevisForbrukMWh"] / tot_per_time

# 10) Norsk tittel
tittel = (
    f"Gjennomsnittlig timevis forbruk per prisområde\n"
    f"{span_start} til {span_end}"
)

# 11) Lag linjeplottet
fig = px.line(
    avg_hourly,
    x="timePaaDøgnet",
    y="gjennomsnittligTimevisForbrukMWh",
    color=area_col,
    custom_data=[area_col, "andelAvForbruketDenTimen"],
    markers=True,
    title=tittel,
    labels={
        "timePaaDøgnet": "Time på døgnet",
        "gjennomsnittligTimevisForbrukMWh": "Gjennomsnittlig forbruk (MWh)"
    },
    color_discrete_sequence=elhub_colors
)

# 12) Stil traces og hover-tekst på norsk
fig.update_traces(
    marker=dict(size=MARKER_SIZE),
    line=dict(width=3),
    hovertemplate=(
        "Forbruk: %{y:.2f} MWh<br>"
        "Time på døgnet: %{x}<br>"
        "Prisområde: %{customdata[0]}<br>"
        "Andel av forbruket den timen: %{customdata[1]:.1%}"
        "<extra></extra>"
    )
)

# 13) Layout-innstillinger
fig.update_layout(
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    font=dict(family=FONT_FAMILY),
    xaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS),
        dtick=1
    ),
    yaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    legend_title_text="Prisområde",
    legend=dict(
        title_font_size=AXIS_TITLE_FS,
        font_size=TICK_FS,
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    margin=dict(t=100, b=80),
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# 14) Vis plottet inline
fig.show()

# 15) Lagre i figures/
os.makedirs("figures", exist_ok=True)
output_path = os.path.join("figures", "diurnalt_forbruk_prisområde_MWh.pdf")
fig.write_image(output_path, format="pdf", width=1000, height=700, scale=1)
print(f"✅ Lagret PDF til: {output_path}")


### Daglig produksjon profil - prisområde

In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

# ─── VISNING I NOTEBOOK ─────────────────────────────────────────────────────────
pio.renderers.default = "notebook"  # eller "jupyterlab" hvis du er i JupyterLab
# ──────────────────────────────────────────────────────────────────────────────

# ─── FONT- OG STILINNSTILLINGER ────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
MARKER_SIZE   = 8
FONT_FAMILY   = "Roboto"
# ──────────────────────────────────────────────────────────────────────────────

# Elhub sine farger
elhub_colors = [
    "#212148",  # Mørk lilla
    "#7fb48a",  # Grønn
    "#5369b2",  # Blå
    "#05677d",  # Blå-grønn
    "#886599",  # Lilla
    "#d58000",  # Oker
]

# ─── 0) Klargjør dataframe ─────────────────────────────────────────────────────
df = big_df.copy()

# 1) Finn tidskolonne
time_col = "startTime" if "startTime" in df.columns else "endTime"

# 2) Finn områdekolonne
area_col = "priceArea" if "priceArea" in df.columns else "name"

# 3) Bruk produksjonskolonnen
value_col = "productionQuantityKwh"

# 4) Sørg for korrekt datetime og Oslo-sone
df[time_col] = (
    pd.to_datetime(df[time_col], utc=True)
      .dt.tz_convert("Europe/Oslo")
)

# 5) Dato-span for tittel
span_start = df[time_col].min().date().isoformat()
span_end   = df[time_col].max().date().isoformat()

# 6) Ekstraher time på døgnet
df["timePaaDøgnet"] = df[time_col].dt.hour

# 7) Beregn gjennomsnittlig produksjon per time og område (i kWh)
avg_hourly = (
    df
    .groupby(["timePaaDøgnet", area_col], observed=True)[value_col]
    .mean()
    .reset_index()
    .rename(columns={value_col: "gjennomsnittligTimevisProduksjonKWh"})
)

# 8) Konverter til MWh
avg_hourly["gjennomsnittligTimevisProduksjonMWh"] = (
    avg_hourly["gjennomsnittligTimevisProduksjonKWh"] / 1_000
)

# 9) Andel av totalsum per time
tot_per_time = avg_hourly.groupby("timePaaDøgnet")["gjennomsnittligTimevisProduksjonMWh"].transform("sum")
avg_hourly["andelAvProduksjonenDenTimen"] = avg_hourly["gjennomsnittligTimevisProduksjonMWh"] / tot_per_time

# 10) Norsk tittel
tittel = (
    f"Gjennomsnittlig timevis produksjon per prisområde\n"
    f"{span_start} til {span_end}"
)

# 11) Lag linjeplottet
fig = px.line(
    avg_hourly,
    x="timePaaDøgnet",
    y="gjennomsnittligTimevisProduksjonMWh",
    color=area_col,
    custom_data=[area_col, "andelAvProduksjonenDenTimen"],
    markers=True,
    title=tittel,
    labels={
        "timePaaDøgnet": "Time på døgnet",
        "gjennomsnittligTimevisProduksjonMWh": "Gjennomsnittlig produksjon (MWh)"
    },
    color_discrete_sequence=elhub_colors
)

# 12) Stil traces og hover-tekst på norsk
fig.update_traces(
    marker=dict(size=MARKER_SIZE),
    line=dict(width=3),
    hovertemplate=(
        "Produksjon: %{y:.2f} MWh<br>"
        "Time på døgnet: %{x}<br>"
        "Prisområde: %{customdata[0]}<br>"
        "Andel av produksjonen den timen: %{customdata[1]:.1%}"
        "<extra></extra>"
    )
)

# 13) Layout-innstillinger
fig.update_layout(
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    font=dict(family=FONT_FAMILY),
    xaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS),
        dtick=1
    ),
    yaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    legend_title_text="Prisområde",
    legend=dict(
        title_font_size=AXIS_TITLE_FS,
        font_size=TICK_FS,
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    margin=dict(t=100, b=80),
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# 14) Vis plottet inline
fig.show()

# 15) Lagre i figures/
os.makedirs("figures", exist_ok=True)
output_path = os.path.join("figures", "diurnalt_produksjon_prisområde_MWh.pdf")
fig.write_image(output_path, format="pdf", width=1000, height=700, scale=1)
print(f"✅ Lagret PDF til: {output_path}")


### Daglig import profil - prisområde

In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

# ─── VISNING I NOTEBOOK ─────────────────────────────────────────────────────────
pio.renderers.default = "notebook"  # eller "jupyterlab" hvis du er i JupyterLab
# ──────────────────────────────────────────────────────────────────────────────

# ─── FONT- OG STILINNSTILLINGER ────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
MARKER_SIZE   = 8
FONT_FAMILY   = "Roboto"
# ──────────────────────────────────────────────────────────────────────────────

# Elhub sine farger
elhub_colors = [
    "#212148",  # Mørk lilla
    "#7fb48a",  # Grønn
    "#5369b2",  # Blå
    "#05677d",  # Blå-grønn
    "#886599",  # Lilla
    "#d58000",  # Oker
]

# ─── 0) Klargjør dataframe ─────────────────────────────────────────────────────
df = big_df.copy()

# 1) Finn tidskolonne
time_col = "startTime" if "startTime" in df.columns else "endTime"

# 2) Finn områdekolonne
area_col = "priceArea" if "priceArea" in df.columns else "name"

# 3) Bruk kolonnen for innkommende utveksling
value_col = "exchangeInQuantityKwh"

# 4) Sørg for korrekt datetime og Oslo-sone
df[time_col] = (
    pd.to_datetime(df[time_col], utc=True)
      .dt.tz_convert("Europe/Oslo")
)

# 5) Dato-span for tittel
span_start = df[time_col].min().date().isoformat()
span_end   = df[time_col].max().date().isoformat()

# 6) Ekstraher time på døgnet
df["timePaaDøgnet"] = df[time_col].dt.hour

# 7) Beregn gjennomsnittlig innkommende utveksling per time og område (i kWh)
avg_hourly = (
    df
    .groupby(["timePaaDøgnet", area_col], observed=True)[value_col]
    .mean()
    .reset_index()
    .rename(columns={value_col: "gjennomsnittligTimevisImportKWh"})
)

# 8) Konverter til MWh
avg_hourly["gjennomsnittligTimevisImportMWh"] = (
    avg_hourly["gjennomsnittligTimevisImportKWh"] / 1_000
)

# 9) Andel av totalsum per time
tot_per_time = avg_hourly.groupby("timePaaDøgnet")["gjennomsnittligTimevisImportMWh"].transform("sum")
avg_hourly["andelAvImportenDenTimen"] = avg_hourly["gjennomsnittligTimevisImportMWh"] / tot_per_time

# 10) Norsk tittel
tittel = (
    f"Gjennomsnittlig timevis import per prisområde\n"
    f"{span_start} til {span_end}"
)

# 11) Lag linjeplottet
fig = px.line(
    avg_hourly,
    x="timePaaDøgnet",
    y="gjennomsnittligTimevisImportMWh",
    color=area_col,
    custom_data=[area_col, "andelAvImportenDenTimen"],
    markers=True,
    title=tittel,
    labels={
        "timePaaDøgnet": "Time på døgnet",
        "gjennomsnittligTimevisImportMWh": "Gjennomsnittlig import (MWh)"
    },
    color_discrete_sequence=elhub_colors
)

# 12) Stil traces og hover-tekst på norsk
fig.update_traces(
    marker=dict(size=MARKER_SIZE),
    line=dict(width=3),
    hovertemplate=(
        "Import: %{y:.2f} MWh<br>"
        "Time på døgnet: %{x}<br>"
        "Prisområde: %{customdata[0]}<br>"
        "Andel av importen den timen: %{customdata[1]:.1%}"
        "<extra></extra>"
    )
)

# 13) Layout-innstillinger
fig.update_layout(
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    font=dict(family=FONT_FAMILY),
    xaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS),
        dtick=1
    ),
    yaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    legend_title_text="Prisområde",
    legend=dict(
        title_font_size=AXIS_TITLE_FS,
        font_size=TICK_FS,
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    margin=dict(t=100, b=80),
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# 14) Vis plottet inline
fig.show()

# 15) Lagre i figures/
os.makedirs("figures", exist_ok=True)
output_path = os.path.join("figures", "diurnalt_import_prisområde_MWh.pdf")
fig.write_image(output_path, format="pdf", width=1000, height=700, scale=1)
print(f"✅ Lagret PDF til: {output_path}")


### Daglig eksport profil - prisområde

In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

# ─── VISNING I NOTEBOOK ─────────────────────────────────────────────────────────
pio.renderers.default = "notebook"  # eller "jupyterlab" hvis du er i JupyterLab
# ──────────────────────────────────────────────────────────────────────────────

# ─── FONT- OG STILINNSTILLINGER ────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
MARKER_SIZE   = 8
FONT_FAMILY   = "Roboto"
# ──────────────────────────────────────────────────────────────────────────────

# Elhub sine farger
elhub_colors = [
    "#212148",  # Mørk lilla
    "#7fb48a",  # Grønn
    "#5369b2",  # Blå
    "#05677d",  # Blå-grønn
    "#886599",  # Lilla
    "#d58000",  # Oker
]

# ─── 0) Klargjør dataframe ─────────────────────────────────────────────────────
df = big_df.copy()

# 1) Finn tidskolonne
time_col = "startTime" if "startTime" in df.columns else "endTime"

# 2) Finn områdekolonne
area_col = "priceArea" if "priceArea" in df.columns else "name"

# 3) Bruk kolonnen for utgående utveksling (eksport)
value_col = "exchangeOutQuantityKwh"

# 4) Sørg for korrekt datetime og Oslo-sone
df[time_col] = (
    pd.to_datetime(df[time_col], utc=True)
      .dt.tz_convert("Europe/Oslo")
)

# 5) Dato-span for tittel
span_start = df[time_col].min().date().isoformat()
span_end   = df[time_col].max().date().isoformat()

# 6) Ekstraher time på døgnet
df["timePaaDøgnet"] = df[time_col].dt.hour

# 7) Beregn gjennomsnittlig eksport per time og område (i kWh)
avg_hourly = (
    df
    .groupby(["timePaaDøgnet", area_col], observed=True)[value_col]
    .mean()
    .reset_index()
    .rename(columns={value_col: "gjennomsnittligTimevisEksportKWh"})
)

# 8) Konverter til MWh
avg_hourly["gjennomsnittligTimevisEksportMWh"] = (
    avg_hourly["gjennomsnittligTimevisEksportKWh"] / 1_000
)

# 9) Andel av totalsum per time
tot_per_time = avg_hourly.groupby("timePaaDøgnet")["gjennomsnittligTimevisEksportMWh"].transform("sum")
avg_hourly["andelAvEksportenDenTimen"] = avg_hourly["gjennomsnittligTimevisEksportMWh"] / tot_per_time

# 10) Norsk tittel
tittel = (
    f"Gjennomsnittlig timevis eksport per prisområde\n"
    f"{span_start} til {span_end}"
)

# 11) Lag linjeplottet
fig = px.line(
    avg_hourly,
    x="timePaaDøgnet",
    y="gjennomsnittligTimevisEksportMWh",
    color=area_col,
    custom_data=[area_col, "andelAvEksportenDenTimen"],
    markers=True,
    title=tittel,
    labels={
        "timePaaDøgnet": "Time på døgnet",
        "gjennomsnittligTimevisEksportMWh": "Gjennomsnittlig eksport (MWh)"
    },
    color_discrete_sequence=elhub_colors
)

# 12) Stil traces og hover-tekst på norsk
fig.update_traces(
    marker=dict(size=MARKER_SIZE),
    line=dict(width=3),
    hovertemplate=(
        "Eksport: %{y:.2f} MWh<br>"
        "Time på døgnet: %{x}<br>"
        "Prisområde: %{customdata[0]}<br>"
        "Andel av eksporten den timen: %{customdata[1]:.1%}"
        "<extra></extra>"
    )
)

# 13) Layout-innstillinger
fig.update_layout(
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    font=dict(family=FONT_FAMILY),
    xaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS),
        dtick=1
    ),
    yaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    legend_title_text="Prisområde",
    legend=dict(
        title_font_size=AXIS_TITLE_FS,
        font_size=TICK_FS,
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    margin=dict(t=100, b=80),
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# 14) Vis plottet inline
fig.show()

# 15) Lagre i figures/
os.makedirs("figures", exist_ok=True)
output_path = os.path.join("figures", "diurnalt_eksport_prisområde_MWh.pdf")
fig.write_image(output_path, format="pdf", width=1000, height=700, scale=1)
print(f"✅ Lagret PDF til: {output_path}")


### Daglig net infeed profil - prisområde

In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

# ─── VISNING I NOTEBOOK ─────────────────────────────────────────────────────────
pio.renderers.default = "notebook"  # eller "jupyterlab" hvis du er i JupyterLab
# ──────────────────────────────────────────────────────────────────────────────

# ─── FONT- OG STILINNSTILLINGER ────────────────────────────────────────────────
TITLE_FS      = 24
AXIS_TITLE_FS = 20
TICK_FS       = 18
MARKER_SIZE   = 8
FONT_FAMILY   = "Roboto"
# ──────────────────────────────────────────────────────────────────────────────

# Elhub sine farger
elhub_colors = [
    "#212148",  # Mørk lilla
    "#7fb48a",  # Grønn
    "#5369b2",  # Blå
    "#05677d",  # Blå-grønn
    "#886599",  # Lilla
    "#d58000",  # Oker
]

# ─── 0) Klargjør dataframe ─────────────────────────────────────────────────────
df = big_df.copy()

# 1) Finn tidskolonne
time_col = "startTime" if "startTime" in df.columns else "endTime"

# 2) Finn områdekolonne
area_col = "priceArea" if "priceArea" in df.columns else "name"

# 3) Bruk kolonnen for netto innmating (net infeed)
value_col = "netInfeedQuantityKwh"

# 4) Sørg for korrekt datetime og Oslo-sone
df[time_col] = (
    pd.to_datetime(df[time_col], utc=True)
      .dt.tz_convert("Europe/Oslo")
)

# 5) Dato-span for tittel
span_start = df[time_col].min().date().isoformat()
span_end   = df[time_col].max().date().isoformat()

# 6) Ekstraher time på døgnet
df["timePaaDøgnet"] = df[time_col].dt.hour

# 7) Beregn gjennomsnittlig netto innmating per time og område (i kWh)
avg_hourly = (
    df
    .groupby(["timePaaDøgnet", area_col], observed=True)[value_col]
    .mean()
    .reset_index()
    .rename(columns={value_col: "gjennomsnittligTimevisInnmatningKWh"})
)

# 8) Konverter til MWh
avg_hourly["gjennomsnittligTimevisInnmatningMWh"] = (
    avg_hourly["gjennomsnittligTimevisInnmatningKWh"] / 1_000
)

# 9) Andel av totalsum per time
tot_per_time = avg_hourly.groupby("timePaaDøgnet")["gjennomsnittligTimevisInnmatningMWh"].transform("sum")
avg_hourly["andelAvInnmatningenDenTimen"] = avg_hourly["gjennomsnittligTimevisInnmatningMWh"] / tot_per_time

# 10) Norsk tittel
tittel = (
    f"Gjennomsnittlig timevis netto innmating per prisområde\n"
    f"{span_start} til {span_end}"
)

# 11) Lag linjeplottet
fig = px.line(
    avg_hourly,
    x="timePaaDøgnet",
    y="gjennomsnittligTimevisInnmatningMWh",
    color=area_col,
    custom_data=[area_col, "andelAvInnmatningenDenTimen"],
    markers=True,
    title=tittel,
    labels={
        "timePaaDøgnet": "Time på døgnet",
        "gjennomsnittligTimevisInnmatningMWh": "Gjennomsnittlig netto innmating (MWh)"
    },
    color_discrete_sequence=elhub_colors
)

# 12) Stil traces og hover-tekst på norsk
fig.update_traces(
    marker=dict(size=MARKER_SIZE),
    line=dict(width=3),
    hovertemplate=(
        "Innmating: %{y:.2f} MWh<br>"
        "Time på døgnet: %{x}<br>"
        "Prisområde: %{customdata[0]}<br>"
        "Andel av innmatningen den timen: %{customdata[1]:.1%}"
        "<extra></extra>"
    )
)

# 13) Layout-innstillinger
fig.update_layout(
    title_font=dict(size=TITLE_FS, family=FONT_FAMILY),
    title_x=0.5,
    font=dict(family=FONT_FAMILY),
    xaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS),
        dtick=1
    ),
    yaxis=dict(
        title_font=dict(size=AXIS_TITLE_FS),
        tickfont=dict(size=TICK_FS)
    ),
    legend_title_text="Prisområde",
    legend=dict(
        title_font_size=AXIS_TITLE_FS,
        font_size=TICK_FS,
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    margin=dict(t=100, b=80),
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# 14) Vis plottet inline
fig.show()

# 15) Lagre i figures/
os.makedirs("figures", exist_ok=True)
output_path = os.path.join("figures", "diurnalt_netto_innmating_prisområde_MWh.pdf")
fig.write_image(output_path, format="pdf", width=1000, height=700, scale=1)
print(f"✅ Lagret PDF til: {output_path}")
